
# Glue Studio Notebook
You are now running a **Glue Studio** notebook; before you can start using your notebook you *must* start an interactive session.

## Available Magics
|          Magic              |   Type       |                                                                        Description                                                                        |
|-----------------------------|--------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| %%configure                 |  Dictionary  |  A json-formatted dictionary consisting of all configuration parameters for a session. Each parameter can be specified here or through individual magics. |
| %profile                    |  String      |  Specify a profile in your aws configuration to use as the credentials provider.                                                                          |
| %iam_role                   |  String      |  Specify an IAM role to execute your session with.                                                                                                        |
| %region                     |  String      |  Specify the AWS region in which to initialize a session.                                                                                                 |
| %session_id                 |  String      |  Returns the session ID for the running session.                                                                                                          |
| %connections                |  List        |  Specify a comma separated list of connections to use in the session.                                                                                     |
| %additional_python_modules  |  List        |  Comma separated list of pip packages, s3 paths or private pip arguments.                                                                                 |
| %extra_py_files             |  List        |  Comma separated list of additional Python files from S3.                                                                                                 |
| %extra_jars                 |  List        |  Comma separated list of additional Jars to include in the cluster.                                                                                       |
| %number_of_workers          |  Integer     |  The number of workers of a defined worker_type that are allocated when a job runs. worker_type must be set too.                                          |
| %glue_version               |  String      |  The version of Glue to be used by this session. Currently, the only valid options are 2.0 and 3.0 (eg: %glue_version 2.0).                               |
| %security_config            |  String      |  Define a security configuration to be used with this session.                                                                                            |
| %sql                        |  String      |  Run SQL code. All lines after the initial %%sql magic will be passed as part of the SQL code.                                                            |
| %streaming                  |  String      |  Changes the session type to Glue Streaming.                                                                                                              |
| %etl                        |  String      |  Changes the session type to Glue ETL.                                                                                                                    |
| %status                     |              |  Returns the status of the current Glue session including its duration, configuration and executing user / role.                                          |
| %stop_session               |              |  Stops the current session.                                                                                                                               |
| %list_sessions              |              |  Lists all currently running sessions by name and ID.                                                                                                     |
| %worker_type                |  String      |  Standard, G.1X, *or* G.2X. number_of_workers must be set too. Default is G.1X.                                                                           |
| %spark_conf                 |  String      |  Specify custom spark configurations for your session. E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer.                      |

In [10]:
%stop_session

Stopping session: f9980bcb-e0c0-463f-9055-66d35acb6116
Stopped session.


# Step 1 define Settings

In [8]:
%connections hudi-connection
%glue_version 3.0
%region us-west-2
%worker_type G.1X
%number_of_workers 3
%spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer
%additional_python_modules Faker

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.37.0 
Connections to be included:
hudi-connection
Setting Glue version to: 3.0
Previous region: us-west-2
Setting new region to: us-west-2
Reauthenticating Glue client with new region: us-west-2
IAM role has been set to arn:aws:iam::043916019468:role/Lab3. Reauthenticating.
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::043916019468:role/Lab3
Authentication done.
Region is set to: us-west-2
Previous worker type: G.1X
Setting new worker type to: G.1X
Previous number of workers: 5
Setting new number of workers to: 3
Previous Spark configuration: None
Setting new Spark configuration to: spark.serializer=org.ap

# Step 2: Define Imports

In [1]:
try:
    import sys
    from awsglue.transforms import *
    from awsglue.utils import getResolvedOptions
    from pyspark.context import SparkContext
    from awsglue.context import GlueContext
    from awsglue.job import Job
    from pyspark.sql.session import SparkSession
    from awsglue.dynamicframe import DynamicFrame
    from pyspark.sql.functions import col, to_timestamp, monotonically_increasing_id, to_date, when
    from pyspark.sql.functions import *
    from awsglue.utils import getResolvedOptions
    from pyspark.sql.types import *
    from datetime import datetime
    import boto3
    from functools import reduce
except Exception as e:
    pass

Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 3
Session ID: 9c22bfce-8c32-4b35-a234-e8dca21cbea7
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
--conf spark.serializer=org.apache.spark.serializer.KryoSerializer
--additional-python-modules Faker
Waiting for session 9c22bfce-8c32-4b35-a234-e8dca21cbea7 to get into ready status...
Session 9c22bfce-8c32-4b35-a234-e8dca21cbea7 has been created.



# Step 3: Create Spark Session

In [2]:
spark = SparkSession.builder.config('spark.serializer','org.apache.spark.serializer.KryoSerializer').config('spark.sql.hive.convertMetastoreParquet','false').config('spark.sql.legacy.pathOptionBehavior.enabled', 'true') .getOrCreate()

sc = spark.sparkContext
glueContext = GlueContext(sc)
job = Job(glueContext)
logger = glueContext.get_logger()

# Creating Hudi Tables 

In [39]:
db_name = "hudidb"
table_name="hudi_table"

recordkey = 'emp_id'
precombine = 'ts'

path = "s3://soumilshah-hudi-demos/tmp/"

method = 'upsert'
table_type = "COPY_ON_WRITE"

connection_options={
    "path": path,
    "connectionName": "hudi-connection",

    "hoodie.datasource.write.storage.type": table_type,
    'className': 'org.apache.hudi',
    'hoodie.table.name': table_name,
    'hoodie.datasource.write.recordkey.field': recordkey,
    'hoodie.datasource.write.table.name': table_name,
    'hoodie.datasource.write.operation': method,
    'hoodie.datasource.write.precombine.field': precombine,


    'hoodie.datasource.hive_sync.enable': 'true',
    "hoodie.datasource.hive_sync.mode":"hms",
    'hoodie.datasource.hive_sync.sync_as_datasource': 'false',
    'hoodie.datasource.hive_sync.database': db_name,
    'hoodie.datasource.hive_sync.table': table_name,
    'hoodie.datasource.hive_sync.use_jdbc': 'false',
    'hoodie.datasource.hive_sync.partition_extractor_class': 'org.apache.hudi.hive.MultiPartKeysValueExtractor',
    'hoodie.datasource.write.hive_style_partitioning': 'true',
}

In [54]:
impleDataUpd = [
    (1, "This is APPEND 1", "Sales", "RJ", 81000, 30, 23000, 827307999),
    (2, "This is APPEND 2", "Engineering", "RJ", 79000, 53, 15000, 1627694678),
]

columns = ["emp_id", "employee_name", "department", "state", "salary", "age", "bonus", "ts"]
spark_df = spark.createDataFrame(data=impleDataUpd, schema=columns)



WriteDF = (
    glueContext.write_dynamic_frame.from_options(
        frame=DynamicFrame.fromDF(spark_df, glueContext,"glue_df"),
        connection_type="marketplace.spark",
        connection_options=connection_options,
        transformation_ctx="glue_df",
    )
)

spark.sql(f"select * from {db_name}.{table_name} ").show(truncate=False)


+-------------------+---------------------+------------------+----------------------+---------------------------------------------------------------------------+------+----------------+-----------+-----+------+---+-----+----------+
|_hoodie_commit_time|_hoodie_commit_seqno |_hoodie_record_key|_hoodie_partition_path|_hoodie_file_name                                                          |emp_id|employee_name   |department |state|salary|age|bonus|ts        |
+-------------------+---------------------+------------------+----------------------+---------------------------------------------------------------------------+------+----------------+-----------+-----+------+---+-----+----------+
|20230106235917496  |20230106235917496_0_5|1                 |                      |ba6886a1-2ff5-4f4b-90c0-83b8aa1689c1-0_0-150-2519_20230106235917496.parquet|1     |This is APPEND 1|Sales      |RJ   |81000 |30 |23000|827307999 |
|20230106235917496  |20230106235917496_0_6|2                 |          

# Incremental Data  Processing 

In [55]:
spark. \
      read. \
      format("hudi"). \
      load(path). \
      createOrReplaceTempView("hudi_snapshot")

commits = list(map(lambda row: row[0], spark.sql("select distinct(_hoodie_commit_time) as commitTime from  hudi_snapshot order by commitTime").limit(50).collect()))
beginTime = commits[len(commits) - 2] # commit time we are interested in

print(f"commits : {commits} beginTime : {beginTime} ")

incremental_read_options = {
  'hoodie.datasource.query.type': 'incremental',
  'hoodie.datasource.read.begin.instanttime': beginTime,
}

IncrementalDF = spark.read.format("hudi"). \
  options(**incremental_read_options). \
  load(path)

IncrementalDF.createOrReplaceTempView("hudi_incremental")
spark.sql("select * from  hudi_incremental").show()


commits : ['20230106235917496'] beginTime : 20230106235917496 
+-------------------+--------------------+------------------+----------------------+-----------------+------+-------------+----------+-----+------+---+-----+---+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|_hoodie_file_name|emp_id|employee_name|department|state|salary|age|bonus| ts|
+-------------------+--------------------+------------------+----------------------+-----------------+------+-------------+----------+-----+------+---+-----+---+
+-------------------+--------------------+------------------+----------------------+-----------------+------+-------------+----------+-----+------+---+-----+---+


## Inserting Sample New records and then updating one records 
### Goal is to see new data that was added and updated records


In [56]:
impleDataUpd = [
    (1, "This is UPDATE 1", "Sales", "RJ", 81000, 30, 23000, 827307999),
    (3, "This is NEW APPEND 2", "Engineering", "RJ", 79000, 53, 15000, 1627694678),
]

columns = ["emp_id", "employee_name", "department", "state", "salary", "age", "bonus", "ts"]
spark_df = spark.createDataFrame(data=impleDataUpd, schema=columns)
spark_df.show()
WriteDF = (
    glueContext.write_dynamic_frame.from_options(
        frame=DynamicFrame.fromDF(spark_df, glueContext,"glue_df"),
        connection_type="marketplace.spark",
        connection_options=connection_options,
        transformation_ctx="glue_df",
    )
)

spark.sql(f"select * from {db_name}.{table_name} ").show(truncate=False)


+------+--------------------+-----------+-----+------+---+-----+----------+
|emp_id|       employee_name| department|state|salary|age|bonus|        ts|
+------+--------------------+-----------+-----+------+---+-----+----------+
|     1|    This is UPDATE 1|      Sales|   RJ| 81000| 30|23000| 827307999|
|     3|This is NEW APPEND 2|Engineering|   RJ| 79000| 53|15000|1627694678|
+------+--------------------+-----------+-----+------+---+-----+----------+

+-------------------+---------------------+------------------+----------------------+---------------------------------------------------------------------------+------+--------------------+-----------+-----+------+---+-----+----------+
|_hoodie_commit_time|_hoodie_commit_seqno |_hoodie_record_key|_hoodie_partition_path|_hoodie_file_name                                                          |emp_id|employee_name       |department |state|salary|age|bonus|ts        |
+-------------------+---------------------+------------------+---------

# Running Incremental Query to See if we see updated data and new incoming data 

In [57]:
spark. \
      read. \
      format("hudi"). \
      load(path). \
      createOrReplaceTempView("hudi_snapshot")

commits = list(map(lambda row: row[0], spark.sql("select distinct(_hoodie_commit_time) as commitTime from  hudi_snapshot order by commitTime").limit(50).collect()))
beginTime = commits[len(commits) - 2] # commit time we are interested in

print(f"commits : {commits} beginTime : {beginTime} ")

incremental_read_options = {
  'hoodie.datasource.query.type': 'incremental',
  'hoodie.datasource.read.begin.instanttime': beginTime,
}

IncrementalDF = spark.read.format("hudi"). \
  options(**incremental_read_options). \
  load(path)

IncrementalDF.createOrReplaceTempView("hudi_incremental")
spark.sql("select * from  hudi_incremental").show()


commits : ['20230106235917496', '20230107000130345'] beginTime : 20230106235917496 
+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------------+-----------+-----+------+---+-----+----------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|emp_id|       employee_name| department|state|salary|age|bonus|        ts|
+-------------------+--------------------+------------------+----------------------+--------------------+------+--------------------+-----------+-----+------+---+-----+----------+
|  20230107000130345|20230107000130345...|                 1|                      |ba6886a1-2ff5-4f4...|     1|    This is UPDATE 1|      Sales|   RJ| 81000| 30|23000| 827307999|
|  20230107000130345|20230107000130345...|                 3|                      |ba6886a1-2ff5-4f4...|     3|This is NEW APPEND 2|Engineering|   RJ| 79000| 53|15000|1627694678|
+---------------